In [86]:
#Import libraries
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.stats.outliers_influence import variance_inflation_factor 

In [87]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC 
from sklearn.impute import KNNImputer


In [88]:
#Import tensor flow libraries
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

In [89]:
kepler_df = pd.read_csv("Resources/kepler_dataset.csv")
pd.set_option('display.max_columns', None)

kepler_df.head()

,kepid,kepoi_name,kepler_name,koi_disposition,koi_vet_stat,koi_vet_date,koi_pdisposition,koi_score,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_disp_prov,koi_comment,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,koi_time0,koi_time0_err1,koi_time0_err2,koi_eccen,koi_eccen_err1,koi_eccen_err2,koi_longp,koi_longp_err1,koi_longp_err2,koi_impact,koi_impact_err1,koi_impact_err2,koi_duration,koi_duration_err1,koi_duration_err2,koi_ingress,koi_ingress_err1,koi_ingress_err2,koi_depth,koi_depth_err1,koi_depth_err2,koi_ror,koi_ror_err1,koi_ror_err2,koi_srho,koi_srho_err1,koi_srho_err2,koi_fittype,koi_prad,koi_prad_err1,koi_prad_err2,koi_sma,koi_sma_err1,koi_sma_err2,koi_incl,koi_incl_err1,koi_incl_err2,koi_teq,koi_teq_err1,koi_teq_err2,koi_insol,koi_insol_err1,koi_insol_err2,koi_dor,koi_dor_err1,koi_dor_err2,koi_limbdark_mod,koi_ldm_coeff4,koi_ldm_coeff3,koi_ldm_coeff2,koi_ldm_coeff1,koi_parm_prov,koi_max_sngle_ev,koi_max_mult_ev,koi_model_snr,koi_count,koi_num_transits,koi_tce_plnt_num,koi_tce_delivname,koi_quarters,koi_bin_oedp_sig,koi_trans_mod,koi_model_dof,koi_model_chisq,koi_datalink_dvr,koi_datalink_dvs,koi_steff,koi_steff_err1,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_smet,koi_smet_err1,koi_smet_err2,koi_srad,koi_srad_err1,koi_srad_err2,koi_smass,koi_smass_err1,koi_smass_err2,koi_sage,koi_sage_err1,koi_sage_err2,koi_sparprov,ra,dec,koi_kepmag,koi_gmag,koi_rmag,koi_imag,koi_zmag,koi_jmag,koi_hmag,koi_kmag,koi_fwm_stat_sig,koi_fwm_sra,koi_fwm_sra_err,koi_fwm_sdec,koi_fwm_sdec_err,koi_fwm_srao,koi_fwm_srao_err,koi_fwm_sdeco,koi_fwm_sdeco_err,koi_fwm_prao,koi_fwm_prao_err,koi_fwm_pdeco,koi_fwm_pdeco_err,koi_dicco_mra,koi_dicco_mra_err,koi_dicco_mdec,koi_dicco_mdec_err,koi_dicco_msky,koi_dicco_msky_err,koi_dikco_mra,koi_dikco_mra_err,koi_dikco_mdec,koi_dikco_mdec_err,koi_dikco_msky,koi_dikco_msky_err
0,10797460,K00752.01,Kepler-227 b,CONFIRMED,Done,2018-08-16,CANDIDATE,1.000,0,0,0,0,q1_q17_dr25_sup_koi,NO_COMMENT,9.488036,2.780000e-05,-2.780000e-05,170.538750,0.002160,-0.002160,2455003.539,0.002160,-0.002160,0.0,NaN,NaN,NaN,NaN,NaN,0.146,0.318,-0.146,2.95750,0.08190,-0.08190,NaN,NaN,NaN,616.0,19.5,-19.5,0.022344,0.000832,-0.000528,3.20796,0.33173,-1.09986,LS+MCMC,2.26,0.26,-0.15,0.0853,NaN,NaN,89.66,NaN,NaN,793.0,NaN,NaN,93.59,29.45,-16.65,24.810,2.600,-2.600,Claret (2011 A&A 529 75) ATLAS LS,0.0,0.0,0.2291,0.4603,q1_q17_dr25_koi,5.135849,28.470820,35.8,2,142.0,1.0,q1_q17_dr25_tce,11111111111111111000000000000000,0.6864,Mandel and Agol (2002 ApJ 580 171),NaN,NaN,010/010797/010797460/dv/kplr010797460-20160209...,010/010797/010797460/dv/kplr010797460-001-2016...,5455.0,81.0,-81.0,4.467,0.064,-0.096,0.14,0.15,-0.15,0.927,0.105,-0.061,0.919,0.052,-0.046,NaN,NaN,NaN,q1_q17_dr25_stellar,291.93423,48.141651,15.347,15.890,15.270,15.114,15.006,14.082,13.751,13.648,0.002,19.462294,1.400000e-05,48.14191,0.000130,0.430,0.510,0.940,0.480,-0.00020,0.00032,-0.00055,0.00031,-0.010,0.130,0.200,0.160,0.200,0.170,0.080,0.130,0.310,0.170,0.320,0.160
1,10797460,K00752.02,Kepler-227 c,CONFIRMED,Done,2018-08-16,CANDIDATE,0.969,0,0,0,0,q1_q17_dr25_sup_koi,NO_COMMENT,54.418383,2.480000e-04,-2.480000e-04,162.513840,0.003520,-0.003520,2454995.514,0.003520,-0.003520,0.0,NaN,NaN,NaN,NaN,NaN,0.586,0.059,-0.443,4.50700,0.11600,-0.11600,NaN,NaN,NaN,875.0,35.5,-35.5,0.027954,0.009080,-0.001350,3.02368,2.20489,-2.49638,LS+MCMC,2.83,0.32,-0.19,0.2734,NaN,NaN,89.57,NaN,NaN,443.0,NaN,NaN,9.11,2.87,-1.62,77.900,28.400,-28.400,Claret (2011 A&A 529 75) ATLAS LS,0.0,0.0,0.2291,0.4603,q1_q17_dr25_koi,7.027669,20.109507,25.8,2,25.0,2.0,q1_q17_dr25_tce,11111111111111111000000000000000,0.0023,Mandel and Agol (2002 ApJ 580 171),NaN,NaN,010/010797/010797460/dv/kplr010797460-20160209...,010/010797/010797460/dv/kplr010797460-002-2016...,5455.0,81.0,-81.0,4.467,0.064,-0.096,0.14,0.15,-0.15,0.927,0.105,-0.061,0.919,0.052,-0.046,NaN,NaN,NaN,q1_q17_dr25_stellar,291.93423,48.141651,15.347,15.890,15.270,15.114,15.0

In [90]:
# Documentation link 
# https://exoplanetarchive.ipac.caltech.edu/docs/API_kepcandidate_columns.html

# Exploration and Preprocessing

In [91]:
kepler_df.dtypes
#print(len(kepler_df.columns))

kepid                   int64
kepoi_name             object
kepler_name            object
koi_disposition        object
koi_vet_stat           object
                       ...   
koi_dikco_mra_err     float64
koi_dikco_mdec        float64
koi_dikco_mdec_err    float64
koi_dikco_msky        float64
koi_dikco_msky_err    float64
Length: 140, dtype: object

In [92]:
kepler_df.shape

(9564, 140)

In [93]:
kepler_df.isna().sum()/len(kepler_df)

kepid                 0.000000
kepoi_name            0.000000
kepler_name           0.712986
koi_disposition       0.000000
koi_vet_stat          0.000000
                        ...   
koi_dikco_mra_err     0.059598
koi_dikco_mdec        0.059598
koi_dikco_mdec_err    0.059598
koi_dikco_msky        0.059598
koi_dikco_msky_err    0.059598
Length: 140, dtype: float64

In [94]:
kepler_df.isnull().any(axis=1).sum()

9564

In [95]:
# Check if any column is completely NaN 
empty_columns = kepler_df.isnull().all() 
print(empty_columns) 
# Get a list of all completely empty columns
completely_nan_columns = empty_columns[empty_columns].index.tolist() 
print(f'Columns that are completely NaN: {completely_nan_columns}')

kepid                 False
kepoi_name            False
kepler_name           False
koi_disposition       False
koi_vet_stat          False
                      ...  
koi_dikco_mra_err     False
koi_dikco_mdec        False
koi_dikco_mdec_err    False
koi_dikco_msky        False
koi_dikco_msky_err    False
Length: 140, dtype: bool
Columns that are completely NaN: ['koi_eccen_err1', 'koi_eccen_err2', 'koi_longp', 'koi_longp_err1', 'koi_longp_err2', 'koi_ingress', 'koi_ingress_err1', 'koi_ingress_err2', 'koi_sma_err1', 'koi_sma_err2', 'koi_incl_err1', 'koi_incl_err2', 'koi_teq_err1', 'koi_teq_err2', 'koi_model_dof', 'koi_model_chisq', 'koi_sage', 'koi_sage_err1', 'koi_sage_err2']


In [96]:
print(kepler_df['koi_disp_prov'].nunique())
print(kepler_df['koi_vet_stat'].nunique())
print(kepler_df['koi_limbdark_mod'].nunique())
print(kepler_df['koi_trans_mod'].nunique())

1
1
1
1


In [97]:
#remove empty columns and columns with only one value
kepler_df_cleaned = kepler_df.drop(columns =['koi_eccen_err1', 'koi_eccen_err2', 'koi_longp', 'koi_longp_err1', 'koi_longp_err2', 'koi_ingress', 
                                     'koi_ingress_err1', 'koi_ingress_err2', 'koi_sma_err1', 'koi_sma_err2', 'koi_incl_err1', 
                                     'koi_incl_err2', 'koi_teq_err1', 'koi_teq_err2', 'koi_model_dof', 'koi_model_chisq', 'koi_sage', 
                                     'koi_sage_err1', 'koi_sage_err2', 'koi_disp_prov', 'koi_vet_stat', 'koi_limbdark_mod', 'koi_trans_mod'])
display(kepler_df_cleaned.head())
print(kepler_df_cleaned.shape)

,kepid,kepoi_name,kepler_name,koi_disposition,koi_vet_date,koi_pdisposition,koi_score,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_comment,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,koi_time0,koi_time0_err1,koi_time0_err2,koi_eccen,koi_impact,koi_impact_err1,koi_impact_err2,koi_duration,koi_duration_err1,koi_duration_err2,koi_depth,koi_depth_err1,koi_depth_err2,koi_ror,koi_ror_err1,koi_ror_err2,koi_srho,koi_srho_err1,koi_srho_err2,koi_fittype,koi_prad,koi_prad_err1,koi_prad_err2,koi_sma,koi_incl,koi_teq,koi_insol,koi_insol_err1,koi_insol_err2,koi_dor,koi_dor_err1,koi_dor_err2,koi_ldm_coeff4,koi_ldm_coeff3,koi_ldm_coeff2,koi_ldm_coeff1,koi_parm_prov,koi_max_sngle_ev,koi_max_mult_ev,koi_model_snr,koi_count,koi_num_transits,koi_tce_plnt_num,koi_tce_delivname,koi_quarters,koi_bin_oedp_sig,koi_datalink_dvr,koi_datalink_dvs,koi_steff,koi_steff_err1,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_smet,koi_smet_err1,koi_smet_err2,koi_srad,koi_srad_err1,koi_srad_err2,koi_smass,koi_smass_err1,koi_smass_err2,koi_sparprov,ra,dec,koi_kepmag,koi_gmag,koi_rmag,koi_imag,koi_zmag,koi_jmag,koi_hmag,koi_kmag,koi_fwm_stat_sig,koi_fwm_sra,koi_fwm_sra_err,koi_fwm_sdec,koi_fwm_sdec_err,koi_fwm_srao,koi_fwm_srao_err,koi_fwm_sdeco,koi_fwm_sdeco_err,koi_fwm_prao,koi_fwm_prao_err,koi_fwm_pdeco,koi_fwm_pdeco_err,koi_dicco_mra,koi_dicco_mra_err,koi_dicco_mdec,koi_dicco_mdec_err,koi_dicco_msky,koi_dicco_msky_err,koi_dikco_mra,koi_dikco_mra_err,koi_dikco_mdec,koi_dikco_mdec_err,koi_dikco_msky,koi_dikco_msky_err
0,10797460,K00752.01,Kepler-227 b,CONFIRMED,2018-08-16,CANDIDATE,1.000,0,0,0,0,NO_COMMENT,9.488036,2.780000e-05,-2.780000e-05,170.538750,0.002160,-0.002160,2455003.539,0.002160,-0.002160,0.0,0.146,0.318,-0.146,2.95750,0.08190,-0.08190,616.0,19.5,-19.5,0.022344,0.000832,-0.000528,3.20796,0.33173,-1.09986,LS+MCMC,2.26,0.26,-0.15,0.0853,89.66,793.0,93.59,29.45,-16.65,24.810,2.600,-2.600,0.0,0.0,0.2291,0.4603,q1_q17_dr25_koi,5.135849,28.470820,35.8,2,142.0,1.0,q1_q17_dr25_tce,11111111111111111000000000000000,0.6864,010/010797/010797460/dv/kplr010797460-20160209...,010/010797/010797460/dv/kplr010797460-001-2016...,5455.0,81.0,-81.0,4.467,0.064,-0.096,0.14,0.15,-0.15,0.927,0.105,-0.061,0.919,0.052,-0.046,q1_q17_dr25_stellar,291.93423,48.141651,15.347,15.890,15.270,15.114,15.006,14.082,13.751,13.648,0.002,19.462294,1.400000e-05,48.14191,0.000130,0.430,0.510,0.940,0.480,-0.00020,0.00032,-0.00055,0.00031,-0.010,0.130,0.200,0.160,0.200,0.170,0.080,0.130,0.310,0.170,0.320,0.160
1,10797460,K00752.02,Kepler-227 c,CONFIRMED,2018-08-16,CANDIDATE,0.969,0,0,0,0,NO_COMMENT,54.418383,2.480000e-04,-2.480000e-04,162.513840,0.003520,-0.003520,2454995.514,0.003520,-0.003520,0.0,0.586,0.059,-0.443,4.50700,0.11600,-0.11600,875.0,35.5,-35.5,0.027954,0.009080,-0.001350,3.02368,2.20489,-2.49638,LS+MCMC,2.83,0.32,-0.19,0.2734,89.57,443.0,9.11,2.87,-1.62,77.900,28.400,-28.400,0.0,0.0,0.2291,0.4603,q1_q17_dr25_koi,7.027669,20.109507,25.8,2,25.0,2.0,q1_q17_dr25_tce,11111111111111111000000000000000,0.0023,010/010797/010797460/dv/kplr010797460-20160209...,010/010797/010797460/dv/kplr010797460-002-2016...,5455.0,81.0,-81.0,4.467,0.064,-0.096,0.14,0.15,-0.15,0.927,0.105,-0.061,0.919,0.052,-0.046,q1_q17_dr25_stellar,291.93423,48.141651,15.347,15.890,15.270,15.114,15.006,14.082,13.751,13.648,0.003,19.462265,2.000000e-05,48.14199,0.000190,-0.630,0.720,1.230,0.680,0.00066,0.00065,-0.00105,0.00063,0.390,0.360,0.000,0.480,0.390,0.360,0.490,0.340,0.120,0.730,0.500,0.450
2,10811496,K00753.01,NaN,CANDIDATE,2018-08-16,CANDIDATE,0.000,0,0,0,0,DEEP_V_SHAPED,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,-0.000581,2455008.850,0.000581,-0.000581,0.0,0.969,5.126,-0.077,1.78220,0.03410,-0.03410,10800.0,171.0,-171.0,0.154046,5.030000,-0.042200,7.29555,35.03293,-2.75453,LS+MCMC,14.60,3.92,-1.31,0.1419,88.96,638.0,39.30,31.04,-10.49,53.500,25.700,-25.700,0.0,0.0,0.2711,0.3858,q1_q17_dr25_koi,37.159767,187.449100,76.3,1,56.0

(9564, 117)


In [98]:
#pd.set_option('display.max_rows', None)
print((kepler_df_cleaned.isna().sum()/len(kepler_df)).sort_values(ascending=False))
print(kepler_df_cleaned.isnull().any(axis=1).sum())

kepler_name         0.712986
koi_score           0.157884
koi_bin_oedp_sig    0.157884
koi_comment         0.126412
koi_max_sngle_ev    0.119406
                      ...   
koi_duration        0.000000
koi_fittype         0.000000
koi_parm_prov       0.000000
kepoi_name          0.000000
koi_count           0.000000
Length: 117, dtype: float64
7086


In [99]:
print(f"before: {kepler_df_cleaned['koi_comment'].isna().sum()}")

before: 1209


In [100]:
# if the comment column is left empty it is logical to assume there is no comment
kepler_df_cleaned['koi_comment'] = kepler_df_cleaned['koi_comment'].fillna('NO_COMMENT')

print(f"after: {kepler_df_cleaned['koi_comment'].isna().sum()}")

after: 0


In [101]:
kepler_df_cleaned['koi_disposition'].value_counts()

koi_disposition
FALSE POSITIVE    4839
CONFIRMED         2743
CANDIDATE         1982
Name: count, dtype: int64

## split the data

In [102]:
y = kepler_df_cleaned['koi_disposition']


In [103]:
#create X data, drop columns that might cause data leakage and columns from y data
X = kepler_df_cleaned.drop(columns = ['koi_disposition','koi_pdisposition' , 'kepid', 'kepoi_name', 
                                      'koi_vet_date', 'kepler_name', 'koi_quarters', 'koi_datalink_dvr', 'koi_datalink_dvs'])
X.head(3)

,koi_score,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_comment,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,koi_time0,koi_time0_err1,koi_time0_err2,koi_eccen,koi_impact,koi_impact_err1,koi_impact_err2,koi_duration,koi_duration_err1,koi_duration_err2,koi_depth,koi_depth_err1,koi_depth_err2,koi_ror,koi_ror_err1,koi_ror_err2,koi_srho,koi_srho_err1,koi_srho_err2,koi_fittype,koi_prad,koi_prad_err1,koi_prad_err2,koi_sma,koi_incl,koi_teq,koi_insol,koi_insol_err1,koi_insol_err2,koi_dor,koi_dor_err1,koi_dor_err2,koi_ldm_coeff4,koi_ldm_coeff3,koi_ldm_coeff2,koi_ldm_coeff1,koi_parm_prov,koi_max_sngle_ev,koi_max_mult_ev,koi_model_snr,koi_count,koi_num_transits,koi_tce_plnt_num,koi_tce_delivname,koi_bin_oedp_sig,koi_steff,koi_steff_err1,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_smet,koi_smet_err1,koi_smet_err2,koi_srad,koi_srad_err1,koi_srad_err2,koi_smass,koi_smass_err1,koi_smass_err2,koi_sparprov,ra,dec,koi_kepmag,koi_gmag,koi_rmag,koi_imag,koi_zmag,koi_jmag,koi_hmag,koi_kmag,koi_fwm_stat_sig,koi_fwm_sra,koi_fwm_sra_err,koi_fwm_sdec,koi_fwm_sdec_err,koi_fwm_srao,koi_fwm_srao_err,koi_fwm_sdeco,koi_fwm_sdeco_err,koi_fwm_prao,koi_fwm_prao_err,koi_fwm_pdeco,koi_fwm_pdeco_err,koi_dicco_mra,koi_dicco_mra_err,koi_dicco_mdec,koi_dicco_mdec_err,koi_dicco_msky,koi_dicco_msky_err,koi_dikco_mra,koi_dikco_mra_err,koi_dikco_mdec,koi_dikco_mdec_err,koi_dikco_msky,koi_dikco_msky_err
0,1.000,0,0,0,0,NO_COMMENT,9.488036,0.000028,-0.000028,170.538750,0.002160,-0.002160,2455003.539,0.002160,-0.002160,0.0,0.146,0.318,-0.146,2.9575,0.0819,-0.0819,616.0,19.5,-19.5,0.022344,0.000832,-0.000528,3.20796,0.33173,-1.09986,LS+MCMC,2.26,0.26,-0.15,0.0853,89.66,793.0,93.59,29.45,-16.65,24.81,2.6,-2.6,0.0,0.0,0.2291,0.4603,q1_q17_dr25_koi,5.135849,28.470820,35.8,2,142.0,1.0,q1_q17_dr25_tce,0.6864,5455.0,81.0,-81.0,4.467,0.064,-0.096,0.14,0.15,-0.15,0.927,0.105,-0.061,0.919,0.052,-0.046,q1_q17_dr25_stellar,291.93423,48.141651,15.347,15.890,15.27,15.114,15.006,14.082,13.751,13.648,0.002,19.462294,0.000014,48.14191,0.00013,0.430,0.510,0.940,0.480,-0.00020,0.00032,-0.00055,0.00031,-0.010,0.13,0.200,0.16,0.200,0.170,0.080,0.130,0.310,0.170,0.320,0.160
1,0.969,0,0,0,0,NO_COMMENT,54.418383,0.000248,-0.000248,162.513840,0.003520,-0.003520,2454995.514,0.003520,-0.003520,0.0,0.586,0.059,-0.443,4.5070,0.1160,-0.1160,875.0,35.5,-35.5,0.027954,0.009080,-0.001350,3.02368,2.20489,-2.49638,LS+MCMC,2.83,0.32,-0.19,0.2734,89.57,443.0,9.11,2.87,-1.62,77.90,28.4,-28.4,0.0,0.0,0.2291,0.4603,q1_q17_dr25_koi,7.027669,20.109507,25.8,2,25.0,2.0,q1_q17_dr25_tce,0.0023,5455.0,81.0,-81.0,4.467,0.064,-0.096,0.14,0.15,-0.15,0.927,0.105,-0.061,0.919,0.052,-0.046,q1_q17_dr25_stellar,291.93423,48.141651,15.347,15.890,15.27,15.114,15.006,14.082,13.751,13.648,0.003,19.462265,0.000020,48.14199,0.00019,-0.630,0.720,1.230,0.680,0.00066,0.00065,-0.00105,0.00063,0.390,0.36,0.000,0.48,0.390,0.360,0.490,0.340,0.120,0.730,0.500,0.450
2,0.000,0,0,0,0,DEEP_V_SHAPED,19.899140,0.000015,-0.000015,175.850252,0.000581,-0.000581,2455008.850,0.000581,-0.000581,0.0,0.969,5.126,-0.077,1.7822,0.0341,-0.0341,10800.0,171.0,-171.0,0.154046,5.030000,-0.042200,7.29555,35.03293,-2.75453,LS+MCMC,14.60,3.92,-1.31,0.1419,88.96,638.0,39.30,31.04,-10.49,53.50,25.7,-25.7,0.0,0.0,0.2711,0.3858,q1_q17_dr25_koi,37.159767,187.449100,76.3,1,56.0,1.0,q1_q17_dr25_tce,0.6624,5853.0,158.0,-176.0,4.544,0.044,-0.176,-0.18,0.30,-0.30,0.868,0.233,-0.078,0.961,0.110,-0.121,q1_q17_dr25_stellar,297.00482,48.134129,15.436,15.943,15.39,15.220,15.166,14.254,13.900,13.826,0.278,19.800321,0.000002,48.13412,0.00002,-0.021,0.069,-0.038,0.071,0.00070,0.00240,0.00060,0.00340,-0.025,0.07,-0.034,0.07,0.042,0.072,0.002,0.071,-0.027,0.074,0.027,0.074


In [104]:
X['koi_comment'].value_counts()

koi_comment
NO_COMMENT                                                                                                             4318
CENT_KIC_POS                                                                                                            514
CENT_RESOLVED_OFFSET                                                                                                    173
MOD_SEC_DV---MOD_SEC_ALT---HAS_SEC_TCE                                                                                  165
CENT_FEW_DIFFS                                                                                                          161
                                                                                                                       ... 
MOD_ODDEVEN_DV---MOD_ODDEVEN_ALT---DEEP_V_SHAPED---SEASONAL_DEPTH_ALT---CENT_KIC_POS                                      1
MOD_SEC_DV---MOD_SEC_ALT---HAS_SEC_TCE---SEASONAL_DEPTH_DV---SEASONAL_DEPTH_ALT---CENT_KIC_POS---HALO_GHOST             

In [105]:
from sklearn.feature_extraction.text import TfidfVectorizer
#use tf-idf to make numeric values for comments 

comments = X['koi_comment'] 

# Apply TF-IDF Vectorization
vectorizer = TfidfVectorizer() 
comments_tfidf = vectorizer.fit_transform(comments)

# Convert to DataFrame if needed
import pandas as pd
comments_df = pd.DataFrame(comments_tfidf.toarray(), columns=vectorizer.get_feature_names_out())
comments_df.head()


,01,all_trans_chases,alt_robo_odd_even_test_fail,alt_sec_same_depth_as_pri_could_be_twice_true_period,alt_sig_pri_minus_sig_pos_too_low,alt_sig_pri_minus_sig_ter_too_low,alt_sig_pri_over_fred_too_low,cent_crowded,cent_few_diffs,cent_few_meas,cent_kic_pos,cent_nofits,cent_resolved_offset,cent_saturated,cent_uncertain,cent_unresolved_offset,centroid_signif_uncertain,clear_apo,crowded_diff,cyg,deep_v_shaped,depth_oddeven_alt,depth_oddeven_dv,dv_sig_pri_minus_sig_pos_too_low,dv_sig_pri_over_fred_too_low,ephem_match,eyeball,fit_failed,halo_ghost,has_sec_tce,inconsistent_trans,indiv_trans_chases,indiv_trans_chases_marshall,indiv_trans_chases_marshall_skye,indiv_trans_chases_marshall_zuma,indiv_trans_chases_skye,indiv_trans_marshall,indiv_trans_marshall_skye,indiv_trans_marshall_zuma,indiv_trans_rubble,indiv_trans_rubble_marshall_skye,indiv_trans_rubble_skye,indiv_trans_rubble_skye_zuma,indiv_trans_rubble_skye_zuma_tracker,indiv_trans_skye,indiv_trans_skye_zuma,indiv_trans_skye_zuma_tracker,indiv_trans_zuma,invert_diff,is_sec_tce,kic_offset,lpp_alt,lpp_alt_too_high,lpp_dv,lpp_dv_too_high,lyr,marshall_fail,mod_nonuniq_alt,mod_nonuniq_dv,mod_oddeven_alt,mod_oddeven_dv,mod_pos_alt,mod_pos_dv,mod_sec_alt,mod_sec_dv,mod_ter_alt,mod_ter_dv,no_comment,other_tce_at_same_period_diff_epoch,parent_is_002305372,parent_is_002449084,parent_is_003352751,parent_is_003858884,parent_is_004482641,parent_is_005024292,parent_is_005036538,parent_is_005343976,parent_is_005471619,parent_is_005513861,parent_is_006367628,parent_is_007258889,parent_is_007598128,parent_is_008265951,parent_is_008380743,parent_is_009541127,parent_is_009777062,parent_is_010485137,parent_is_010858720,parent_is_012004679,parent_is_3597,parent_is_3895,parent_is_4673,parent_is_489,parent_is_5335,parent_is_970,parent_is_fl,parent_is_rr,parent_is_uz,parent_is_v2277,parent_is_v380,parent_is_v850,period_alias_alt,period_alias_dv,period_alias_in_alt_data_seen_at_3,period_alias_in_dv_data_seen_at_3,planet_in_star,planet_occult_alt,planet_occult_dv,planet_period_is_half_alt,planet_period_is_half_dv,pri,resid_of_prev_tce,residual_tce,same_ntl_period,same_p_as_prev_ntl_tce,saturated,seasonal_depth_alt,seasonal_depth_diffs_in_alt,seasonal_depth_dv,sec,sig_sec_in_alt_model_shift,sig_sec_in_dv_model_shift,signif_offset,sweet_eb,sweet_ntl,too_few_centroids,too_few_quarters,trans_gapped,transits_not_consistent
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.

In [106]:
# Sum the TF-IDF values for each comment to get a single value per comment
comments_df['aggregated_tfidf'] = comments_df.sum(axis=1)

# add aggregated_tfidf back into X to replace koi_comment with numeric values
X['koi_comment_vectorized'] = comments_df['aggregated_tfidf']

# Verify the result
X[['koi_comment_vectorized', 'koi_comment']].head(10)


,koi_comment_vectorized,koi_comment
0,1.000000,NO_COMMENT
1,1.000000,NO_COMMENT
2,1.000000,DEEP_V_SHAPED
3,1.725236,MOD_ODDEVEN_DV---MOD_ODDEVEN_ALT---DEEP_V_SHAPED
4,1.000000,NO_COMMENT
5,1.000000,NO_COMMENT
6,1.000000,NO_COMMENT
7,1.000000,NO_COMMENT
8,1.998752,MOD_SEC_DV---MOD_SEC_ALT---HAS_SEC_TCE---CENT_...
9,1.000000,NO_COMMENT


In [107]:
#drop koi_comment not that it has been replaced
X = X.drop(columns=['koi_comment'])
X.head(3)

,koi_score,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,koi_time0,koi_time0_err1,koi_time0_err2,koi_eccen,koi_impact,koi_impact_err1,koi_impact_err2,koi_duration,koi_duration_err1,koi_duration_err2,koi_depth,koi_depth_err1,koi_depth_err2,koi_ror,koi_ror_err1,koi_ror_err2,koi_srho,koi_srho_err1,koi_srho_err2,koi_fittype,koi_prad,koi_prad_err1,koi_prad_err2,koi_sma,koi_incl,koi_teq,koi_insol,koi_insol_err1,koi_insol_err2,koi_dor,koi_dor_err1,koi_dor_err2,koi_ldm_coeff4,koi_ldm_coeff3,koi_ldm_coeff2,koi_ldm_coeff1,koi_parm_prov,koi_max_sngle_ev,koi_max_mult_ev,koi_model_snr,koi_count,koi_num_transits,koi_tce_plnt_num,koi_tce_delivname,koi_bin_oedp_sig,koi_steff,koi_steff_err1,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_smet,koi_smet_err1,koi_smet_err2,koi_srad,koi_srad_err1,koi_srad_err2,koi_smass,koi_smass_err1,koi_smass_err2,koi_sparprov,ra,dec,koi_kepmag,koi_gmag,koi_rmag,koi_imag,koi_zmag,koi_jmag,koi_hmag,koi_kmag,koi_fwm_stat_sig,koi_fwm_sra,koi_fwm_sra_err,koi_fwm_sdec,koi_fwm_sdec_err,koi_fwm_srao,koi_fwm_srao_err,koi_fwm_sdeco,koi_fwm_sdeco_err,koi_fwm_prao,koi_fwm_prao_err,koi_fwm_pdeco,koi_fwm_pdeco_err,koi_dicco_mra,koi_dicco_mra_err,koi_dicco_mdec,koi_dicco_mdec_err,koi_dicco_msky,koi_dicco_msky_err,koi_dikco_mra,koi_dikco_mra_err,koi_dikco_mdec,koi_dikco_mdec_err,koi_dikco_msky,koi_dikco_msky_err,koi_comment_vectorized
0,1.000,0,0,0,0,9.488036,0.000028,-0.000028,170.538750,0.002160,-0.002160,2455003.539,0.002160,-0.002160,0.0,0.146,0.318,-0.146,2.9575,0.0819,-0.0819,616.0,19.5,-19.5,0.022344,0.000832,-0.000528,3.20796,0.33173,-1.09986,LS+MCMC,2.26,0.26,-0.15,0.0853,89.66,793.0,93.59,29.45,-16.65,24.81,2.6,-2.6,0.0,0.0,0.2291,0.4603,q1_q17_dr25_koi,5.135849,28.470820,35.8,2,142.0,1.0,q1_q17_dr25_tce,0.6864,5455.0,81.0,-81.0,4.467,0.064,-0.096,0.14,0.15,-0.15,0.927,0.105,-0.061,0.919,0.052,-0.046,q1_q17_dr25_stellar,291.93423,48.141651,15.347,15.890,15.27,15.114,15.006,14.082,13.751,13.648,0.002,19.462294,0.000014,48.14191,0.00013,0.430,0.510,0.940,0.480,-0.00020,0.00032,-0.00055,0.00031,-0.010,0.13,0.200,0.16,0.200,0.170,0.080,0.130,0.310,0.170,0.320,0.160,1.0
1,0.969,0,0,0,0,54.418383,0.000248,-0.000248,162.513840,0.003520,-0.003520,2454995.514,0.003520,-0.003520,0.0,0.586,0.059,-0.443,4.5070,0.1160,-0.1160,875.0,35.5,-35.5,0.027954,0.009080,-0.001350,3.02368,2.20489,-2.49638,LS+MCMC,2.83,0.32,-0.19,0.2734,89.57,443.0,9.11,2.87,-1.62,77.90,28.4,-28.4,0.0,0.0,0.2291,0.4603,q1_q17_dr25_koi,7.027669,20.109507,25.8,2,25.0,2.0,q1_q17_dr25_tce,0.0023,5455.0,81.0,-81.0,4.467,0.064,-0.096,0.14,0.15,-0.15,0.927,0.105,-0.061,0.919,0.052,-0.046,q1_q17_dr25_stellar,291.93423,48.141651,15.347,15.890,15.27,15.114,15.006,14.082,13.751,13.648,0.003,19.462265,0.000020,48.14199,0.00019,-0.630,0.720,1.230,0.680,0.00066,0.00065,-0.00105,0.00063,0.390,0.36,0.000,0.48,0.390,0.360,0.490,0.340,0.120,0.730,0.500,0.450,1.0
2,0.000,0,0,0,0,19.899140,0.000015,-0.000015,175.850252,0.000581,-0.000581,2455008.850,0.000581,-0.000581,0.0,0.969,5.126,-0.077,1.7822,0.0341,-0.0341,10800.0,171.0,-171.0,0.154046,5.030000,-0.042200,7.29555,35.03293,-2.75453,LS+MCMC,14.60,3.92,-1.31,0.1419,88.96,638.0,39.30,31.04,-10.49,53.50,25.7,-25.7,0.0,0.0,0.2711,0.3858,q1_q17_dr25_koi,37.159767,187.449100,76.3,1,56.0,1.0,q1_q17_dr25_tce,0.6624,5853.0,158.0,-176.0,4.544,0.044,-0.176,-0.18,0.30,-0.30,0.868,0.233,-0.078,0.961,0.110,-0.121,q1_q17_dr25_stellar,297.00482,48.134129,15.436,15.943,15.39,15.220,15.166,14.254,13.900,13.826,0.278,19.800321,0.000002,48.13412,0.00002,-0.021,0.069,-0.038,0.071,0.00070,0.00240,0.00060,0.00340,-0.025,0.07,-0.034,0.07,0.042,0.072,0.002,0.071,-0.027,0.074,0.027,0.074,1.0


In [108]:
X.dtypes

koi_score                 float64
koi_fpflag_nt               int64
koi_fpflag_ss               int64
koi_fpflag_co               int64
koi_fpflag_ec               int64
                           ...   
koi_dikco_mdec            float64
koi_dikco_mdec_err        float64
koi_dikco_msky            float64
koi_dikco_msky_err        float64
koi_comment_vectorized    float64
Length: 108, dtype: object

In [109]:
X_train,X_test,y_train,y_test = train_test_split(X, y)
X_train.head(3)

,koi_score,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,koi_time0,koi_time0_err1,koi_time0_err2,koi_eccen,koi_impact,koi_impact_err1,koi_impact_err2,koi_duration,koi_duration_err1,koi_duration_err2,koi_depth,koi_depth_err1,koi_depth_err2,koi_ror,koi_ror_err1,koi_ror_err2,koi_srho,koi_srho_err1,koi_srho_err2,koi_fittype,koi_prad,koi_prad_err1,koi_prad_err2,koi_sma,koi_incl,koi_teq,koi_insol,koi_insol_err1,koi_insol_err2,koi_dor,koi_dor_err1,koi_dor_err2,koi_ldm_coeff4,koi_ldm_coeff3,koi_ldm_coeff2,koi_ldm_coeff1,koi_parm_prov,koi_max_sngle_ev,koi_max_mult_ev,koi_model_snr,koi_count,koi_num_transits,koi_tce_plnt_num,koi_tce_delivname,koi_bin_oedp_sig,koi_steff,koi_steff_err1,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_smet,koi_smet_err1,koi_smet_err2,koi_srad,koi_srad_err1,koi_srad_err2,koi_smass,koi_smass_err1,koi_smass_err2,koi_sparprov,ra,dec,koi_kepmag,koi_gmag,koi_rmag,koi_imag,koi_zmag,koi_jmag,koi_hmag,koi_kmag,koi_fwm_stat_sig,koi_fwm_sra,koi_fwm_sra_err,koi_fwm_sdec,koi_fwm_sdec_err,koi_fwm_srao,koi_fwm_srao_err,koi_fwm_sdeco,koi_fwm_sdeco_err,koi_fwm_prao,koi_fwm_prao_err,koi_fwm_pdeco,koi_fwm_pdeco_err,koi_dicco_mra,koi_dicco_mra_err,koi_dicco_mdec,koi_dicco_mdec_err,koi_dicco_msky,koi_dicco_msky_err,koi_dikco_mra,koi_dikco_mra_err,koi_dikco_mdec,koi_dikco_mdec_err,koi_dikco_msky,koi_dikco_msky_err,koi_comment_vectorized
1449,1.000,0,0,0,0,2.800111,0.000004,-0.000004,132.04865,0.00121,-0.00121,2454965.049,0.00121,-0.00121,0.0,0.806,0.096,-0.571,1.0237,0.0459,-0.0459,707.0,28.5,-28.5,0.027766,0.000146,-0.003780,5.75791,20.07703,-3.80140,LS+MCMC,2.13,0.17,-0.20,0.0343,86.54,956.0,197.81,57.46,-54.04,13.360,6.700,-6.700,0.0,0.0,0.1196,0.6106,q1_q17_dr25_koi,3.956329,28.339994,33.5,1,482.0,1.0,q1_q17_dr25_tce,0.2857,4786.0,130.0,-144.0,4.579,0.060,-0.030,-0.2,0.30,-0.30,0.704,0.054,-0.066,0.686,0.079,-0.053,q1_q17_dr25_stellar,291.22433,45.778091,15.229,16.093,15.176,14.859,14.664,13.657,13.127,12.985,0.000,19.414947,0.000008,45.778099,0.000084,-0.33,0.28,0.03,0.30,0.000440,0.000200,0.000430,0.00023,-0.32,0.13,-0.09,0.13,0.33,0.13,-0.10,0.13,0.18,0.13,0.21,0.13,1.0
560,0.997,0,0,0,0,7.410843,0.000031,-0.000031,135.75425,0.00343,-0.00343,2454968.754,0.00343,-0.00343,0.0,0.064,0.384,-0.064,3.2450,0.0965,-0.0965,124.0,5.2,-5.2,0.010073,0.000381,-0.000304,1.87677,0.10718,-0.71958,LS+MCMC,0.94,0.10,-0.08,0.0702,89.79,850.0,123.40,39.43,-26.72,17.600,1.000,-1.000,0.0,0.0,0.2434,0.4341,q1_q17_dr25_koi,4.211921,21.517443,25.6,5,178.0,2.0,q1_q17_dr25_tce,0.8303,5528.0,110.0,-110.0,4.499,0.063,-0.077,-0.2,0.15,-0.15,0.853,0.089,-0.067,0.838,0.056,-0.046,q1_q17_dr25_stellar,283.00296,44.284229,13.404,13.883,13.367,13.198,13.111,12.198,11.819,11.745,NaN,18.866885,0.000016,44.284450,0.000150,0.79,0.63,0.80,0.53,-0.000118,0.000078,NaN,NaN,0.22,0.35,0.11,0.33,0.25,0.35,0.17,0.42,-0.05,0.29,0.18,0.41,1.0
358,0.993,0,0,0,0,1.475323,0.000004,-0.000004,131.83272,0.00257,-0.00257,2454964.833,0.00257,-0.00257,0.0,0.136,0.339,-0.136,2.3155,0.0735,-0.0735,155.0,6.5,-6.5,0.011027,0.000578,-0.000283,1.03038,0.13094,-0.37244,LS+MCMC,5.88,0.85,-1.97,0.0309,88.41,2740.0,13331.51,4802.54,-7977.74,4.913,0.624,-0.624,0.0,0.0,0.1544,0.5780,q1_q17_dr25_koi,3.990197,20.984596,27.2,3,877.0,3.0,q1_q17_dr25_tce,0.6309,4941.0,49.0,-118.0,3.317,0.210,-0.105,0.4,0.05,-0.25,4.885,0.701,-1.636,1.808,0.181,-0.578,q1_q17_dr25_stellar,286.49042,38.374908,14.604,15.155,14.555,14.368,14.268,13.391,13.010,12.923,0.124,19.099340,0.000016,38.374890,0.000200,-0.89,0.69,-0.06,0.73,0.000100,0.000084,-0.000052,0.00009,-0.08,0.23,-0.76,0.29,0.77,0.29,0.02,0.23,-0.80,0.29,0.80,0.29,1.0


In [110]:

print(X_train['koi_fittype'].value_counts())
print('')
print(X_train['koi_parm_prov'].value_counts())
print('')
print(X_train['koi_tce_delivname'].value_counts())
print('')
print(X_train['koi_sparprov'].value_counts())


koi_fittype
LS+MCMC    5921
MCMC        920
none        266
LS           66
Name: count, dtype: int64

koi_parm_prov
q1_q17_dr25_koi    6034
q1_q16_koi          856
q1_q17_dr24_koi     283
Name: count, dtype: int64

koi_tce_delivname
q1_q17_dr25_tce    6034
q1_q16_tce          590
q1_q17_dr24_tce     283
Name: count, dtype: int64

koi_sparprov
q1_q17_dr25_stellar    5922
stellar_q1_q16          721
stellar_q1_q17          185
Solar                    82
Name: count, dtype: int64


### encoding

In [111]:
#initialize one hot encoder
ohe = OneHotEncoder(sparse_output=False, drop='first')

#fit ohe
ohe.fit(X_train[['koi_fittype','koi_parm_prov','koi_tce_delivname','koi_sparprov']])

#trasform train and test data
X_train_encoded_data = ohe.transform(X_train[['koi_fittype','koi_parm_prov','koi_tce_delivname','koi_sparprov']])
X_test_encoded_data = ohe.transform(X_test[['koi_fittype','koi_parm_prov','koi_tce_delivname','koi_sparprov']])


In [112]:
#create dfs from encoded data
X_train_encoded_df = pd.DataFrame(X_train_encoded_data, columns=ohe.get_feature_names_out(
    ['koi_fittype','koi_parm_prov','koi_tce_delivname','koi_sparprov']))

X_test_encoded_df = pd.DataFrame(X_test_encoded_data, columns=ohe.get_feature_names_out(
    ['koi_fittype','koi_parm_prov','koi_tce_delivname','koi_sparprov']))

#display(X_train_encoded_df)
#display(X_test_encoded_df)


In [113]:
#concat dataframes and remove old columns
X_train_encoded = pd.concat([X_train.reset_index(drop=True), X_train_encoded_df],
                            axis=1).drop(columns=['koi_fittype','koi_parm_prov','koi_tce_delivname','koi_sparprov'])

X_test_encoded = pd.concat([X_test.reset_index(drop=True), X_test_encoded_df],
                            axis=1).drop(columns=['koi_fittype','koi_parm_prov','koi_tce_delivname','koi_sparprov'])

display(X_train_encoded)
display(X_test_encoded)
                                      

,koi_score,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,koi_time0,koi_time0_err1,koi_time0_err2,koi_eccen,koi_impact,koi_impact_err1,koi_impact_err2,koi_duration,koi_duration_err1,koi_duration_err2,koi_depth,koi_depth_err1,koi_depth_err2,koi_ror,koi_ror_err1,koi_ror_err2,koi_srho,koi_srho_err1,koi_srho_err2,koi_prad,koi_prad_err1,koi_prad_err2,koi_sma,koi_incl,koi_teq,koi_insol,koi_insol_err1,koi_insol_err2,koi_dor,koi_dor_err1,koi_dor_err2,koi_ldm_coeff4,koi_ldm_coeff3,koi_ldm_coeff2,koi_ldm_coeff1,koi_max_sngle_ev,koi_max_mult_ev,koi_model_snr,koi_count,koi_num_transits,koi_tce_plnt_num,koi_bin_oedp_sig,koi_steff,koi_steff_err1,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_smet,koi_smet_err1,koi_smet_err2,koi_srad,koi_srad_err1,koi_srad_err2,koi_smass,koi_smass_err1,koi_smass_err2,ra,dec,koi_kepmag,koi_gmag,koi_rmag,koi_imag,koi_zmag,koi_jmag,koi_hmag,koi_kmag,koi_fwm_stat_sig,koi_fwm_sra,koi_fwm_sra_err,koi_fwm_sdec,koi_fwm_sdec_err,koi_fwm_srao,koi_fwm_srao_err,koi_fwm_sdeco,koi_fwm_sdeco_err,koi_fwm_prao,koi_fwm_prao_err,koi_fwm_pdeco,koi_fwm_pdeco_err,koi_dicco_mra,koi_dicco_mra_err,koi_dicco_mdec,koi_dicco_mdec_err,koi_dicco_msky,koi_dicco_msky_err,koi_dikco_mra,koi_dikco_mra_err,koi_dikco_mdec,koi_dikco_mdec_err,koi_dikco_msky,koi_dikco_msky_err,koi_comment_vectorized,koi_fittype_LS+MCMC,koi_fittype_MCMC,koi_fittype_none,koi_parm_prov_q1_q17_dr24_koi,koi_parm_prov_q1_q17_dr25_koi,koi_tce_delivname_q1_q17_dr24_tce,koi_tce_delivname_q1_q17_dr25_tce,koi_tce_delivname_nan,koi_sparprov_q1_q17_dr25_stellar,koi_sparprov_stellar_q1_q16,koi_sparprov_stellar_q1_q17,koi_sparprov_nan
0,1.000,0,0,0,0,2.800111,0.000004,-0.000004,132.048650,0.001210,-0.001210,2454965.049,0.001210,-0.001210,0.0,0.8060,0.0960,-0.5710,1.02370,0.04590,-0.04590,707.0,28.5,-28.5,0.027766,0.000146,-0.003780,5.75791,20.07703,-3.80140,2.13,0.17,-0.20,0.0343,86.54,956.0,197.81,57.46,-54.04,13.360,6.700,-6.700,0.0,0.0,0.1196,0.6106,3.956329,28.339994,33.5,1,482.0,1.0,0.2857,4786.0,130.0,-144.0,4.579,0.060,-0.030,-0.20,0.30,-0.30,0.704,0.054,-0.066,0.686,0.079,-0.053,291.22433,45.778091,15.229,16.093,15.176,14.859,14.664,13.657,13.127,12.985,0.000,19.414947,7.500000e-06,45.778099,0.000084,-0.330,0.280,0.030,0.300,0.000440,0.000200,0.000430,0.000230,-0.320,0.130,-0.090,0.130,0.330,0.130,-0.100,0.130,0.180,0.130,0.210,0.130,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
1,0.997,0,0,0,0,7.410843,0.000031,-0.000031,135.754250,0.003430,-0.003430,2454968.754,0.003430,-0.003430,0.0,0.0640,0.3840,-0.0640,3.24500,0.09650,-0.09650,124.0,5.2,-5.2,0.010073,0.000381,-0.000304,1.87677,0.10718,-0.71958,0.94,0.10,-0.08,0.0702,89.79,850.0,123.40,39.43,-26.72,17.600,1.000,-1.000,0.0,0.0,0.2434,0.4341,4.211921,21.517443,25.6,5,178.0,2.0,0.8303,5528.0,110.0,-110.0,4.499,0.063,-0.077,-0.20,0.15,-0.15,0.853,0.089,-0.067,0.838,0.056,-0.046,283.00296,44.284229,13.404,13.883,13.367,13.198,13.111,12.198,11.819,11.745,NaN,18.866885,1.600000e-05,44.284450,0.000150,0.790,0.630,0.800,0.530,-0.000118,0.000078,NaN,NaN,0.220,0.350,0.110,0.330,0.250,0.350,0.170,0.420,-0.050,0.290,0.180,0.410,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
2,0.993,0,0,0,0,1.475323,0.000004,-0.000004,131.832720,0.002570,-0.002570,2454964.833,0.002570,-0.002570,0.0,0.1360,0.3390,-0.1360,2.31550,0.07350,-0.07350,155.0,6.5,-6.5,0.011027,0.000578,-0.000283,1.03038,0.13094,-0.37244,5.88,0.85,-1.97,0.0309,88.41,2740.0,13331.51,4802.54,-7977.74,4.913,0.624,-0.624,0.0,0.0,0.1544,0.5780,3.990197,20.984596,27.2,3,877.0,3.0,0.6309,4941.0,49.0,-118.0,3.317,0.210,-0.105,0.40,0.05,-0.25,4.885,0.701,-1.636,1.808,0.181,-0.578,286.49042,38.374908,14.604,15.155,14.555,14.368,14.268,13.391,13.010,12.923,0.124,19.099340,1.600000e-05,38.374890,0.000200,-0.890,0.690,-0.060,0.730,0.000100,0.000084,-0.000052,0.000090,-0.080,0.230,-0.760,0.290,0.770,0.290,0.020,0.230,-0.800,0.290,0.800,0.290,1.0,1.0,0.0,0.0,0.0,1.0,0.0

,koi_score,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,koi_time0,koi_time0_err1,koi_time0_err2,koi_eccen,koi_impact,koi_impact_err1,koi_impact_err2,koi_duration,koi_duration_err1,koi_duration_err2,koi_depth,koi_depth_err1,koi_depth_err2,koi_ror,koi_ror_err1,koi_ror_err2,koi_srho,koi_srho_err1,koi_srho_err2,koi_prad,koi_prad_err1,koi_prad_err2,koi_sma,koi_incl,koi_teq,koi_insol,koi_insol_err1,koi_insol_err2,koi_dor,koi_dor_err1,koi_dor_err2,koi_ldm_coeff4,koi_ldm_coeff3,koi_ldm_coeff2,koi_ldm_coeff1,koi_max_sngle_ev,koi_max_mult_ev,koi_model_snr,koi_count,koi_num_transits,koi_tce_plnt_num,koi_bin_oedp_sig,koi_steff,koi_steff_err1,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_smet,koi_smet_err1,koi_smet_err2,koi_srad,koi_srad_err1,koi_srad_err2,koi_smass,koi_smass_err1,koi_smass_err2,ra,dec,koi_kepmag,koi_gmag,koi_rmag,koi_imag,koi_zmag,koi_jmag,koi_hmag,koi_kmag,koi_fwm_stat_sig,koi_fwm_sra,koi_fwm_sra_err,koi_fwm_sdec,koi_fwm_sdec_err,koi_fwm_srao,koi_fwm_srao_err,koi_fwm_sdeco,koi_fwm_sdeco_err,koi_fwm_prao,koi_fwm_prao_err,koi_fwm_pdeco,koi_fwm_pdeco_err,koi_dicco_mra,koi_dicco_mra_err,koi_dicco_mdec,koi_dicco_mdec_err,koi_dicco_msky,koi_dicco_msky_err,koi_dikco_mra,koi_dikco_mra_err,koi_dikco_mdec,koi_dikco_mdec_err,koi_dikco_msky,koi_dikco_msky_err,koi_comment_vectorized,koi_fittype_LS+MCMC,koi_fittype_MCMC,koi_fittype_none,koi_parm_prov_q1_q17_dr24_koi,koi_parm_prov_q1_q17_dr25_koi,koi_tce_delivname_q1_q17_dr24_tce,koi_tce_delivname_q1_q17_dr25_tce,koi_tce_delivname_nan,koi_sparprov_q1_q17_dr25_stellar,koi_sparprov_stellar_q1_q16,koi_sparprov_stellar_q1_q17,koi_sparprov_nan
0,0.000,0,0,0,1,2.253552,8.200000e-06,-8.200000e-06,132.479800,0.003510,-0.003510,2454965.480,0.003510,-0.003510,0.0,0.761,0.007,-0.501,3.66000,0.10200,-0.10200,120.0,3.4,-3.4,0.010887,0.001210,-0.000716,0.12742,0.21178,-0.09834,1.70,0.34,-0.32,0.0360,76.45,1812.0,2547.01,1518.55,-1064.39,3.2500,1.3000,-1.3000,0.0,0.0,0.3110,0.2938,5.716754,28.604504,42.1,1,599.0,1.0,0.3367,6526.0,155.0,-214.0,4.217,0.140,-0.140,-0.14,0.25,-0.30,1.426,0.293,-0.263,1.226,0.157,-0.192,297.12173,42.673321,13.121,13.443,13.063,12.983,12.992,12.071,11.862,11.816,0.000,19.808127,0.000007,42.67361,0.000071,0.45,0.26,1.05,0.26,-0.000044,0.000023,-0.000085,0.000022,-0.400,0.250,0.57,0.14,0.70,0.20,-0.340,0.230,0.580,0.140,0.680,0.180,1.000000,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
1,0.000,1,0,1,1,12.425420,3.600000e-04,-3.600000e-04,141.545800,0.026700,-0.026700,2454974.546,0.026700,-0.026700,0.0,0.428,0.234,-0.428,18.56000,2.30000,-2.30000,145.0,7.9,-7.9,0.011191,0.000565,-0.001370,0.01301,0.00370,-0.00736,0.84,0.14,-0.05,0.0961,84.82,659.0,44.57,23.41,-9.59,4.7400,1.3000,-1.3000,0.0,0.0,0.2665,0.3980,3.630458,13.497753,24.9,1,111.0,1.0,0.5811,5571.0,165.0,-148.0,4.643,0.028,-0.105,-0.70,0.30,-0.30,0.691,0.114,-0.045,0.776,0.066,-0.073,297.66727,40.514130,14.667,15.189,14.624,14.444,14.334,13.409,13.011,12.937,0.000,19.844494,0.000018,40.51458,0.000180,0.43,0.73,1.62,0.67,-0.000033,0.000098,-0.000277,0.000089,-1.980,0.420,0.04,0.48,1.99,0.42,-1.960,0.430,-0.130,0.480,1.970,0.420,1.884659,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
2,0.000,0,1,1,1,1.061933,1.250000e-06,-1.250000e-06,133.850441,0.000978,-0.000978,2454966.850,0.000978,-0.000978,0.0,0.713,0.112,-0.216,2.14290,0.03860,-0.03860,134.0,2.0,-2.0,0.011349,0.001430,-0.000425,0.38294,0.29268,-0.15919,2.29,0.48,-0.71,0.0215,75.43,2508.0,9391.15,5840.09,-5394.87,2.8400,1.1000,-1.1000,0.0,0.0,0.3030,0.3166,5.663719,23.225883,80.2,1,1261.0,1.0,0.8469,6134.0,167.0,-167.0,3.975,0.259,-0.111,-0.08,0.30,-0.30,1.851,0.383,-0.575,1.179,0.204,-0.185,288.90253,44.632992,12.953,13.327,12.888,12.793,12.758,11.858,11.603,11.547,0.000,19.260241,0.000011,44.63222,0.000130,2.77,0.44,-2.77,0.48,-0.000144,0.000025,0.000178,0.000027,0.230,0.380,-2.22,0.44,2.23,0.42,0.510,0.520,-1.870,0.390,1.940,0.400,1.995877,1.0,0.0

In [114]:
#initialize one hot encoder
ohe_y = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

#fit/transform ohe to get encoded y train and test
y_train_encoded = ohe_y.fit_transform(y_train.values.reshape(-1,1))
y_test_encoded = ohe_y.transform(y_test.values.reshape(-1,1))

#display the results
print("Encoded y_train:")
print(y_train_encoded)
print("\nEncoded y_test:")
print(y_test_encoded)
                                      

Encoded y_train:
[[0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 ...
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]]

Encoded y_test:
[[0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 ...
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]]


In [115]:
X_train_encoded.dtypes

koi_score                           float64
koi_fpflag_nt                         int64
koi_fpflag_ss                         int64
koi_fpflag_co                         int64
koi_fpflag_ec                         int64
                                     ...   
koi_tce_delivname_nan               float64
koi_sparprov_q1_q17_dr25_stellar    float64
koi_sparprov_stellar_q1_q16         float64
koi_sparprov_stellar_q1_q17         float64
koi_sparprov_nan                    float64
Length: 116, dtype: object

In [116]:
#use knn imputer to handle nulls 
imputer = KNNImputer(n_neighbors=5)

X_train_imputed = imputer.fit_transform(X_train_encoded)
X_test_imputed = imputer.transform(X_test_encoded)


In [117]:
#scale the data
scaler = StandardScaler()

#fit the scaler
scaler.fit(X_train_imputed)

#scale the data
X_train_scaled = scaler.transform(X_train_imputed)
X_test_scaled = scaler.transform(X_test_imputed)



In [118]:
#convert back to dataframes
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns = X_train_encoded.columns)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns = X_test_encoded.columns)

display(X_train_scaled_df.head(3))
display(X_test_scaled_df.head(3))

,koi_score,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,koi_time0,koi_time0_err1,koi_time0_err2,koi_eccen,koi_impact,koi_impact_err1,koi_impact_err2,koi_duration,koi_duration_err1,koi_duration_err2,koi_depth,koi_depth_err1,koi_depth_err2,koi_ror,koi_ror_err1,koi_ror_err2,koi_srho,koi_srho_err1,koi_srho_err2,koi_prad,koi_prad_err1,koi_prad_err2,koi_sma,koi_incl,koi_teq,koi_insol,koi_insol_err1,koi_insol_err2,koi_dor,koi_dor_err1,koi_dor_err2,koi_ldm_coeff4,koi_ldm_coeff3,koi_ldm_coeff2,koi_ldm_coeff1,koi_max_sngle_ev,koi_max_mult_ev,koi_model_snr,koi_count,koi_num_transits,koi_tce_plnt_num,koi_bin_oedp_sig,koi_steff,koi_steff_err1,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_smet,koi_smet_err1,koi_smet_err2,koi_srad,koi_srad_err1,koi_srad_err2,koi_smass,koi_smass_err1,koi_smass_err2,ra,dec,koi_kepmag,koi_gmag,koi_rmag,koi_imag,koi_zmag,koi_jmag,koi_hmag,koi_kmag,koi_fwm_stat_sig,koi_fwm_sra,koi_fwm_sra_err,koi_fwm_sdec,koi_fwm_sdec_err,koi_fwm_srao,koi_fwm_srao_err,koi_fwm_sdeco,koi_fwm_sdeco_err,koi_fwm_prao,koi_fwm_prao_err,koi_fwm_pdeco,koi_fwm_pdeco_err,koi_dicco_mra,koi_dicco_mra_err,koi_dicco_mdec,koi_dicco_mdec_err,koi_dicco_msky,koi_dicco_msky_err,koi_dikco_mra,koi_dikco_mra_err,koi_dikco_mdec,koi_dikco_mdec_err,koi_dikco_msky,koi_dikco_msky_err,koi_comment_vectorized,koi_fittype_LS+MCMC,koi_fittype_MCMC,koi_fittype_none,koi_parm_prov_q1_q17_dr24_koi,koi_parm_prov_q1_q17_dr25_koi,koi_tce_delivname_q1_q17_dr24_tce,koi_tce_delivname_q1_q17_dr25_tce,koi_tce_delivname_nan,koi_sparprov_q1_q17_dr25_stellar,koi_sparprov_stellar_q1_q16,koi_sparprov_stellar_q1_q17,koi_sparprov_nan
0,1.213672,-0.040276,-0.554966,-0.49114,-0.364687,-0.049639,-0.265254,0.265254,-0.497404,-0.402172,0.402172,-0.497399,-0.402172,0.402172,0.0,0.028443,-0.198930,-0.210838,-0.719704,-0.466964,0.466964,-0.278317,-0.022952,0.022952,-0.077195,-0.190892,0.075516,-0.066116,0.030211,0.054914,-0.034118,-0.044381,0.026315,-0.314279,0.265171,-0.145628,-0.044566,-0.063685,0.042809,-0.069792,-0.053918,0.053918,0.0,0.0,-2.118588,1.944791,-0.213519,-0.228608,-0.286337,-0.465968,0.256463,-0.374437,-0.245971,-1.187679,-0.323368,0.261798,0.639124,-0.467773,1.354893,-0.272443,0.933970,-0.563225,-0.172079,-0.359555,0.141959,-0.979468,-0.524143,0.482633,-0.171945,0.541981,0.692226,0.841215,0.686350,0.612227,0.566371,0.505738,0.392531,0.341143,-0.644612,-0.175431,-0.677878,0.545559,-0.660448,-0.002806,-0.692238,-0.000288,-0.660699,0.017170,-0.024688,0.008895,-0.025946,-0.133351,-0.526883,-0.030624,-0.580886,-0.520674,-0.581134,-0.039989,-0.509370,0.093135,-0.564386,-0.547303,-0.556039,-0.680266,0.459838,-0.383574,-0.196244,-0.202667,0.434469,-0.202667,0.434469,-0.196244,0.459615,-0.334288,-0.162708,-0.195092
1,1.207068,-0.040276,-0.554966,-0.49114,-0.364687,-0.046644,-0.261982,0.261982,-0.442886,-0.301514,0.301514,-0.442889,-0.301514,0.301514,0.0,-0.207225,-0.167781,0.221274,-0.367857,-0.387656,0.387656,-0.285305,-0.027979,0.027979,-0.082889,-0.190866,0.078490,-0.141400,-0.243916,0.156807,-0.034658,-0.044581,0.026446,-0.256231,0.481540,-0.269313,-0.044997,-0.063996,0.043088,-0.065338,-0.070971,0.070971,0.0,0.0,-0.173342,0.255948,-0.213177,-0.230325,-0.296393,4.136288,-0.322428,1.164196,0.912961,-0.239471,-0.750256,0.739132,0.448286,-0.444219,0.796300,-0.272443,-1.034785,1.220799,-0.146183,-0.318165,0.141525,-0.543093,-0.795646,0.521751,-1.898594,0.125723,-0.620147,-0.627725,-0.616224,-0.672465,-0.715582,-0.616764,-0.634736,-0.631778,0.209792,-1.927328,-0.168249,0.119728,-0.294632,0.051880,-0.149144,0.037748,-0.305990,0.008137,-0.024700,0.005996,-0.025925,0.097439,-0.182858,0.048152,-0.250914,-0.547864,-0.259912,0.076148,-0.054972,0.002708,-0.301345,-0.557434,-0.148237,-0.680266,0.459838,-0.383574,-0.196244,-0.202667,0.434469,-0.202667,0.434469,-0.196244,0.459615,-0.334288,-0.162708,-0.195092
2,1.198262,-0.040276,-0.554966,-0.49114,-0.364687,-0.050499,-0.265208,0

,koi_score,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,koi_time0,koi_time0_err1,koi_time0_err2,koi_eccen,koi_impact,koi_impact_err1,koi_impact_err2,koi_duration,koi_duration_err1,koi_duration_err2,koi_depth,koi_depth_err1,koi_depth_err2,koi_ror,koi_ror_err1,koi_ror_err2,koi_srho,koi_srho_err1,koi_srho_err2,koi_prad,koi_prad_err1,koi_prad_err2,koi_sma,koi_incl,koi_teq,koi_insol,koi_insol_err1,koi_insol_err2,koi_dor,koi_dor_err1,koi_dor_err2,koi_ldm_coeff4,koi_ldm_coeff3,koi_ldm_coeff2,koi_ldm_coeff1,koi_max_sngle_ev,koi_max_mult_ev,koi_model_snr,koi_count,koi_num_transits,koi_tce_plnt_num,koi_bin_oedp_sig,koi_steff,koi_steff_err1,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_smet,koi_smet_err1,koi_smet_err2,koi_srad,koi_srad_err1,koi_srad_err2,koi_smass,koi_smass_err1,koi_smass_err2,ra,dec,koi_kepmag,koi_gmag,koi_rmag,koi_imag,koi_zmag,koi_jmag,koi_hmag,koi_kmag,koi_fwm_stat_sig,koi_fwm_sra,koi_fwm_sra_err,koi_fwm_sdec,koi_fwm_sdec_err,koi_fwm_srao,koi_fwm_srao_err,koi_fwm_sdeco,koi_fwm_sdeco_err,koi_fwm_prao,koi_fwm_prao_err,koi_fwm_pdeco,koi_fwm_pdeco_err,koi_dicco_mra,koi_dicco_mra_err,koi_dicco_mdec,koi_dicco_mdec_err,koi_dicco_msky,koi_dicco_msky_err,koi_dikco_mra,koi_dikco_mra_err,koi_dikco_mdec,koi_dikco_mdec_err,koi_dikco_msky,koi_dikco_msky_err,koi_comment_vectorized,koi_fittype_LS+MCMC,koi_fittype_MCMC,koi_fittype_none,koi_parm_prov_q1_q17_dr24_koi,koi_parm_prov_q1_q17_dr25_koi,koi_tce_delivname_q1_q17_dr24_tce,koi_tce_delivname_q1_q17_dr25_tce,koi_tce_delivname_nan,koi_sparprov_q1_q17_dr25_stellar,koi_sparprov_stellar_q1_q16,koi_sparprov_stellar_q1_q17,koi_sparprov_nan
0,-0.987742,-0.040276,-0.554966,-0.491140,2.74208,-0.049994,-0.264746,0.264746,-0.491061,-0.297886,0.297886,-0.491058,-0.297886,0.297886,0.0,0.014150,-0.208556,-0.151178,-0.302122,-0.379036,0.379036,-0.285353,-0.028368,0.028368,-0.082627,-0.190777,0.078138,-0.175332,-0.242481,0.177347,-0.034313,-0.043894,0.026185,-0.311530,-0.406572,0.853190,-0.030973,-0.038532,0.032472,-0.080412,-0.070073,0.070073,0.0,0.0,0.888845,-1.086516,-0.211164,-0.228541,-0.275390,-0.465968,0.479259,-0.374437,-0.137441,1.035883,0.210242,-0.720949,-0.224420,0.160338,0.047547,-0.057866,0.277719,-0.563225,-0.046595,-0.076926,0.056401,0.570811,0.396606,-0.294143,1.066624,-0.323150,-0.823655,-0.920183,-0.835119,-0.838755,-0.813812,-0.714474,-0.600965,-0.576070,-0.644612,1.081380,-0.725843,-0.339513,-0.732502,0.035279,-0.723272,0.050097,-0.722387,0.009335,-0.024705,0.003817,-0.025961,-0.167542,-0.339233,0.229338,-0.564387,-0.394923,-0.478927,-0.143222,-0.352681,0.250399,-0.547946,-0.388581,-0.483217,-0.680266,0.459838,-0.383574,-0.196244,-0.202667,0.434469,-0.202667,0.434469,-0.196244,0.459615,-0.334288,-0.162708,-0.195092
1,-0.987742,0.141536,-0.554966,2.036081,2.74208,-0.043386,-0.222094,0.222094,-0.357679,0.753583,-0.753583,-0.357676,0.753583,-0.753583,0.0,-0.091614,-0.184005,-0.088960,2.057991,3.065996,-3.065996,-0.285053,-0.027397,0.027397,-0.082529,-0.190846,0.077578,-0.177552,-0.245337,0.180355,-0.034703,-0.044467,0.026479,-0.214353,0.150662,-0.492180,-0.045453,-0.064272,0.043263,-0.078846,-0.070073,0.070073,0.0,0.0,0.189624,-0.089476,-0.213955,-0.232344,-0.297284,-0.465968,-0.450012,-0.374437,0.382653,-0.184521,0.423686,0.205641,0.791795,-0.719018,0.463521,-2.060584,0.933970,-0.563225,-0.174338,-0.288602,0.151080,-0.721088,-0.677601,0.370866,1.181198,-0.924799,0.288087,0.240346,0.288881,0.291248,0.293966,0.314936,0.301428,0.303482,-0.644612,1.197627,-0.048337,-0.955041,-0.128352,0.034302,0.006026,0.078253,-0.090080,0.009513,-0.024698,0.001925,-0.025956,-0.842818,-0.073396,0.020581,-0.003436,0.043507,-0.157705,-0.840047,-0.039303,-0.028744,0.011016,0.047060,-0.133673,1.309122,0.459838,-0.383574,-0.196244,-0.202667,0.434469,-0.202667,0.434469,-0.196244,0.459615,-0.334288,-0.162708,-0.195092
2,-0.987742,-0.040276,1.801913,2.036081,2.74208,-0.050768,-0.265589,0.265589,

In [119]:
#check for data leakage with VIF
def calc_vif(X):
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    return vif

In [120]:
vif_scores = calc_vif(X_train_scaled_df)
vif_scores = vif_scores.sort_values("VIF", ascending=False)

vif_df = pd.DataFrame(vif_scores)
vif_df

C:\Users\jclib\anaconda3\envs\dev\lib\site-packages\statsmodels\stats\outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)
C:\Users\jclib\anaconda3\envs\dev\lib\site-packages\statsmodels\regression\linear_model.py:1784: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.uncentered_tss


,variables,VIF
19,koi_duration_err1,inf
12,koi_time0_err1,inf
41,koi_dor_err2,inf
107,koi_parm_prov_q1_q17_dr24_koi,inf
108,koi_parm_prov_q1_q17_dr25_koi,inf
...,...,...
78,koi_fwm_stat_sig,1.192291
1,koi_fpflag_nt,1.004792
14,koi_eccen,NaN
42,koi_ldm_coeff4,NaN


In [121]:
#isolate problematic columns
problematic_columns = vif_df[vif_df['VIF'].isin([float('inf'), float('nan')])]

# Get the column names
problematic_column_names = problematic_columns['variables'].tolist()

print(problematic_column_names)

['koi_duration_err1', 'koi_time0_err1', 'koi_dor_err2', 'koi_parm_prov_q1_q17_dr24_koi', 'koi_parm_prov_q1_q17_dr25_koi', 'koi_tce_delivname_q1_q17_dr24_tce', 'koi_depth_err2', 'koi_depth_err1', 'koi_duration_err2', 'koi_tce_delivname_q1_q17_dr25_tce', 'koi_time0_err2', 'koi_dor_err1', 'koi_time0bk_err2', 'koi_period_err1', 'koi_period_err2', 'koi_time0bk_err1', 'koi_eccen', 'koi_ldm_coeff4', 'koi_ldm_coeff3']


In [122]:
#drop problematic columns
X_train_scaled_df.drop(columns = problematic_column_names, inplace = True)
X_test_scaled_df.drop(columns = problematic_column_names, inplace = True)

display(X_train_scaled_df.head(3))
display(X_test_scaled_df.head(3))


,koi_score,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_time0bk,koi_time0,koi_impact,koi_impact_err1,koi_impact_err2,koi_duration,koi_depth,koi_ror,koi_ror_err1,koi_ror_err2,koi_srho,koi_srho_err1,koi_srho_err2,koi_prad,koi_prad_err1,koi_prad_err2,koi_sma,koi_incl,koi_teq,koi_insol,koi_insol_err1,koi_insol_err2,koi_dor,koi_ldm_coeff2,koi_ldm_coeff1,koi_max_sngle_ev,koi_max_mult_ev,koi_model_snr,koi_count,koi_num_transits,koi_tce_plnt_num,koi_bin_oedp_sig,koi_steff,koi_steff_err1,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_smet,koi_smet_err1,koi_smet_err2,koi_srad,koi_srad_err1,koi_srad_err2,koi_smass,koi_smass_err1,koi_smass_err2,ra,dec,koi_kepmag,koi_gmag,koi_rmag,koi_imag,koi_zmag,koi_jmag,koi_hmag,koi_kmag,koi_fwm_stat_sig,koi_fwm_sra,koi_fwm_sra_err,koi_fwm_sdec,koi_fwm_sdec_err,koi_fwm_srao,koi_fwm_srao_err,koi_fwm_sdeco,koi_fwm_sdeco_err,koi_fwm_prao,koi_fwm_prao_err,koi_fwm_pdeco,koi_fwm_pdeco_err,koi_dicco_mra,koi_dicco_mra_err,koi_dicco_mdec,koi_dicco_mdec_err,koi_dicco_msky,koi_dicco_msky_err,koi_dikco_mra,koi_dikco_mra_err,koi_dikco_mdec,koi_dikco_mdec_err,koi_dikco_msky,koi_dikco_msky_err,koi_comment_vectorized,koi_fittype_LS+MCMC,koi_fittype_MCMC,koi_fittype_none,koi_tce_delivname_nan,koi_sparprov_q1_q17_dr25_stellar,koi_sparprov_stellar_q1_q16,koi_sparprov_stellar_q1_q17,koi_sparprov_nan
0,1.213672,-0.040276,-0.554966,-0.49114,-0.364687,-0.049639,-0.497404,-0.497399,0.028443,-0.198930,-0.210838,-0.719704,-0.278317,-0.077195,-0.190892,0.075516,-0.066116,0.030211,0.054914,-0.034118,-0.044381,0.026315,-0.314279,0.265171,-0.145628,-0.044566,-0.063685,0.042809,-0.069792,-2.118588,1.944791,-0.213519,-0.228608,-0.286337,-0.465968,0.256463,-0.374437,-0.245971,-1.187679,-0.323368,0.261798,0.639124,-0.467773,1.354893,-0.272443,0.933970,-0.563225,-0.172079,-0.359555,0.141959,-0.979468,-0.524143,0.482633,-0.171945,0.541981,0.692226,0.841215,0.686350,0.612227,0.566371,0.505738,0.392531,0.341143,-0.644612,-0.175431,-0.677878,0.545559,-0.660448,-0.002806,-0.692238,-0.000288,-0.660699,0.017170,-0.024688,0.008895,-0.025946,-0.133351,-0.526883,-0.030624,-0.580886,-0.520674,-0.581134,-0.039989,-0.509370,0.093135,-0.564386,-0.547303,-0.556039,-0.680266,0.459838,-0.383574,-0.196244,-0.196244,0.459615,-0.334288,-0.162708,-0.195092
1,1.207068,-0.040276,-0.554966,-0.49114,-0.364687,-0.046644,-0.442886,-0.442889,-0.207225,-0.167781,0.221274,-0.367857,-0.285305,-0.082889,-0.190866,0.078490,-0.141400,-0.243916,0.156807,-0.034658,-0.044581,0.026446,-0.256231,0.481540,-0.269313,-0.044997,-0.063996,0.043088,-0.065338,-0.173342,0.255948,-0.213177,-0.230325,-0.296393,4.136288,-0.322428,1.164196,0.912961,-0.239471,-0.750256,0.739132,0.448286,-0.444219,0.796300,-0.272443,-1.034785,1.220799,-0.146183,-0.318165,0.141525,-0.543093,-0.795646,0.521751,-1.898594,0.125723,-0.620147,-0.627725,-0.616224,-0.672465,-0.715582,-0.616764,-0.634736,-0.631778,0.209792,-1.927328,-0.168249,0.119728,-0.294632,0.051880,-0.149144,0.037748,-0.305990,0.008137,-0.024700,0.005996,-0.025925,0.097439,-0.182858,0.048152,-0.250914,-0.547864,-0.259912,0.076148,-0.054972,0.002708,-0.301345,-0.557434,-0.148237,-0.680266,0.459838,-0.383574,-0.196244,-0.196244,0.459615,-0.334288,-0.162708,-0.195092
2,1.198262,-0.040276,-0.554966,-0.49114,-0.364687,-0.050499,-0.500581,-0.500576,-0.184357,-0.172648,0.159909,-0.515087,-0.284933,-0.082582,-0.190845,0.078508,-0.157817,-0.243590,0.168285,-0.032415,-0.042433,0.024391,-0.319776,0.389666,1.936021,0.031430,0.018003,-0.038252,-0.078665,-1.571782,1.632858,-0.213474,-0.230459,-0.294356,1.835160,1.008640,2.702829,0.488629,-0.989603,-2.052263,0.626818,-2.371355,0.709936,0.463521,1.873327,-2.347288,0.031449,0.554582,0.405554,-0.539903,2.241667,0.679914,-2.451232,-1.166159,-1.520884,0.242783,0.217747,0.239198,0.232466,0.239485,0.301088,0.300643,0.292497,-0.129811,-1.184279,-0.168249,-1.565055,-0.017499,-0.030149,-0.056042,-0.004733,0.002453,0.011666,-0.024699,0.004143,-0.025956,-0.030778,-0.370508,-0.29452

,koi_score,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_time0bk,koi_time0,koi_impact,koi_impact_err1,koi_impact_err2,koi_duration,koi_depth,koi_ror,koi_ror_err1,koi_ror_err2,koi_srho,koi_srho_err1,koi_srho_err2,koi_prad,koi_prad_err1,koi_prad_err2,koi_sma,koi_incl,koi_teq,koi_insol,koi_insol_err1,koi_insol_err2,koi_dor,koi_ldm_coeff2,koi_ldm_coeff1,koi_max_sngle_ev,koi_max_mult_ev,koi_model_snr,koi_count,koi_num_transits,koi_tce_plnt_num,koi_bin_oedp_sig,koi_steff,koi_steff_err1,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_smet,koi_smet_err1,koi_smet_err2,koi_srad,koi_srad_err1,koi_srad_err2,koi_smass,koi_smass_err1,koi_smass_err2,ra,dec,koi_kepmag,koi_gmag,koi_rmag,koi_imag,koi_zmag,koi_jmag,koi_hmag,koi_kmag,koi_fwm_stat_sig,koi_fwm_sra,koi_fwm_sra_err,koi_fwm_sdec,koi_fwm_sdec_err,koi_fwm_srao,koi_fwm_srao_err,koi_fwm_sdeco,koi_fwm_sdeco_err,koi_fwm_prao,koi_fwm_prao_err,koi_fwm_pdeco,koi_fwm_pdeco_err,koi_dicco_mra,koi_dicco_mra_err,koi_dicco_mdec,koi_dicco_mdec_err,koi_dicco_msky,koi_dicco_msky_err,koi_dikco_mra,koi_dikco_mra_err,koi_dikco_mdec,koi_dikco_mdec_err,koi_dikco_msky,koi_dikco_msky_err,koi_comment_vectorized,koi_fittype_LS+MCMC,koi_fittype_MCMC,koi_fittype_none,koi_tce_delivname_nan,koi_sparprov_q1_q17_dr25_stellar,koi_sparprov_stellar_q1_q16,koi_sparprov_stellar_q1_q17,koi_sparprov_nan
0,-0.987742,-0.040276,-0.554966,-0.491140,2.74208,-0.049994,-0.491061,-0.491058,0.014150,-0.208556,-0.151178,-0.302122,-0.285353,-0.082627,-0.190777,0.078138,-0.175332,-0.242481,0.177347,-0.034313,-0.043894,0.026185,-0.311530,-0.406572,0.853190,-0.030973,-0.038532,0.032472,-0.080412,0.888845,-1.086516,-0.211164,-0.228541,-0.275390,-0.465968,0.479259,-0.374437,-0.137441,1.035883,0.210242,-0.720949,-0.224420,0.160338,0.047547,-0.057866,0.277719,-0.563225,-0.046595,-0.076926,0.056401,0.570811,0.396606,-0.294143,1.066624,-0.323150,-0.823655,-0.920183,-0.835119,-0.838755,-0.813812,-0.714474,-0.600965,-0.576070,-0.644612,1.081380,-0.725843,-0.339513,-0.732502,0.035279,-0.723272,0.050097,-0.722387,0.009335,-0.024705,0.003817,-0.025961,-0.167542,-0.339233,0.229338,-0.564387,-0.394923,-0.478927,-0.143222,-0.352681,0.250399,-0.547946,-0.388581,-0.483217,-0.680266,0.459838,-0.383574,-0.196244,-0.196244,0.459615,-0.334288,-0.162708,-0.195092
1,-0.987742,0.141536,-0.554966,2.036081,2.74208,-0.043386,-0.357679,-0.357676,-0.091614,-0.184005,-0.088960,2.057991,-0.285053,-0.082529,-0.190846,0.077578,-0.177552,-0.245337,0.180355,-0.034703,-0.044467,0.026479,-0.214353,0.150662,-0.492180,-0.045453,-0.064272,0.043263,-0.078846,0.189624,-0.089476,-0.213955,-0.232344,-0.297284,-0.465968,-0.450012,-0.374437,0.382653,-0.184521,0.423686,0.205641,0.791795,-0.719018,0.463521,-2.060584,0.933970,-0.563225,-0.174338,-0.288602,0.151080,-0.721088,-0.677601,0.370866,1.181198,-0.924799,0.288087,0.240346,0.288881,0.291248,0.293966,0.314936,0.301428,0.303482,-0.644612,1.197627,-0.048337,-0.955041,-0.128352,0.034302,0.006026,0.078253,-0.090080,0.009513,-0.024698,0.001925,-0.025956,-0.842818,-0.073396,0.020581,-0.003436,0.043507,-0.157705,-0.840047,-0.039303,-0.028744,0.011016,0.047060,-0.133673,1.309122,0.459838,-0.383574,-0.196244,-0.196244,0.459615,-0.334288,-0.162708,-0.195092
2,-0.987742,-0.040276,1.801913,2.036081,2.74208,-0.050768,-0.470895,-0.470902,-0.001095,-0.197200,0.091725,-0.542426,-0.285185,-0.082479,-0.190753,0.078387,-0.170376,-0.241370,0.175335,-0.034045,-0.043493,0.025761,-0.334975,-0.474478,1.665313,0.008630,0.035865,-0.011829,-0.080842,0.763142,-0.868354,-0.211235,-0.229895,-0.226893,-0.465968,1.739870,-0.374437,0.948286,0.534942,0.466375,-0.061104,-0.801707,1.094655,0.392211,0.156711,0.933970,-0.563225,0.027271,0.029504,-0.079103,0.435879,0.951417,-0.255025,-0.659569,0.222904,-0.944465,-0.997286,-0.961128,-0.985710,-1.006972,-0.878348,-0.804376,-0.787132,-0.644612,-0.669954,-0.468031,0.218876,-0.405485,0.148557,-0.443966,-0.138598,-0.383100,0.007717,-0.024705,0.006410,-0.025961,0.101713,-0.135946,-0.869593

In [123]:
#drop columns with highest vif scores
X_train_scaled_df.drop(columns = ['koi_time0bk','koi_time0'], inplace = True)
X_test_scaled_df.drop(columns = ['koi_time0bk','koi_time0'], inplace = True)

In [124]:
#try calculating vif again
vif_scores = calc_vif(X_train_scaled_df)
vif_scores = vif_scores.sort_values("VIF", ascending=False)

vif_df = pd.DataFrame(vif_scores)
vif_df

,variables,VIF
12,koi_ror_err1,3398.869202
7,koi_impact_err1,3388.333835
55,koi_rmag,437.975559
53,koi_kepmag,410.458049
73,koi_fwm_pdeco_err,332.077632
...,...,...
70,koi_fwm_prao,1.350317
68,koi_fwm_sdeco,1.343065
66,koi_fwm_srao,1.257967
61,koi_fwm_stat_sig,1.190463


In [125]:
# Define the ANN Model
model = Sequential([
    layers.Input(shape=(X_train_scaled_df.shape[1],), name='input_layer'),  # Input layer
    layers.Dense(64, activation='relu', name='hidden_layer_1'),  # Hidden layer 1
    layers.Dense(32, activation='relu', name='hidden_layer_2'),  # Hidden layer 2
    layers.Dense(32, activation='relu', name='hidden_layer_3'),  # Hidden layer 3
    layers.Dense(3, activation='softmax', name='output')  # Output layer
])

In [126]:
#compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ hidden_layer_1 (Dense)               │ (None, 64)                  │           6,144 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ hidden_layer_2 (Dense)               │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ hidden_layer_3 (Dense)               │ (None, 32)                  │           1,056 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ output (Dense)                       │ (None, 3)                   │              99 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 9,379 (36.64 KB)

 Trainable params: 9,379 (36.64 KB)

 Non-trainable params: 0 (0.00 B)

In [127]:
model.fit(X_train_scaled_df, y_train_encoded, epochs = 100)

Epoch 1/100
225/225 ━━━━━━━━━━━━━━━━━━━━ 1s 885us/step - accuracy: 0.7047 - loss: 0.6786
Epoch 2/100
225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 829us/step - accuracy: 0.8704 - loss: 0.3175
Epoch 3/100
225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 847us/step - accuracy: 0.8850 - loss: 0.2731
Epoch 4/100
225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 874us/step - accuracy: 0.8990 - loss: 0.2356
Epoch 5/100
225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 852us/step - accuracy: 0.9017 - loss: 0.2272
Epoch 6/100
225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 858us/step - accuracy: 0.9164 - loss: 0.1986
Epoch 7/100
225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 849us/step - accuracy: 0.9215 - loss: 0.1989
Epoch 8/100
225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 823us/step - accuracy: 0.9339 - loss: 0.1699
Epoch 9/100
225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 857us/step - accuracy: 0.9410 - loss: 0.1522
Epoch 10/100
225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 852us/step - accuracy: 0.9472 - loss: 0.1473
Epoch 11/100
225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 854us/step - accuracy: 0.9479 - loss: 0.1357
Epoch 12/100
225/22

In [128]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = model.evaluate(X_test_scaled_df, y_test_encoded, verbose = 2)

# Display the model loss and accuracy results
print(f"Model Loss: {round(model_loss,5)} | Model Accuracy: {round(model_accuracy,5)}")

75/75 - 0s - 2ms/step - accuracy: 0.9130 - loss: 0.7891
Model Loss: 0.78913 | Model Accuracy: 0.91301
